In [1]:
!pip install transformers einops accelerate langchain bitsandbytes sentence_transformers pypdf python-dotenv

In [2]:
!pip install typing-extensions==4.5.0 llama-index

  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
INFO: pip is looking at multiple versions of pydantic to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 1.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.6/174.6 kB 3.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.6/158.6 kB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 kB 3.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.4/156.4 kB 3.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.8/148.8 kB 3.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.6/145.6 kB 3.0 MB/s eta 0:00:0000:01
INFO: pip is still looking at multiple versions of pydantic to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━

In [68]:
!pip install PyPDF2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00a 0:00:01


In [2]:
import logging
import sys
import torch
from pprint import pprint
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# from llama_index import LangchainEmbedding
from llama_index.prompts.prompts import SimpleInputPrompt

AttributeError: module 'pyarrow' has no attribute '__version__'

In [4]:
# !git config --global credential.helper store
# !huggingface-cli login

In [24]:
from pathlib import Path
from llama_index import download_loader

PDFReader = download_loader("PDFReader")


In [27]:
#question creation prompts
q_system_prompt = "You are an expert user extracting questions from a text. You will be passed a piece of text, write a list of questions that can be answered based *solely* on the given text."
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

#answer prompt
a_system_prompt = "You are a data extractor. Extract the exact data from given document. If no information found please reply 'no information available'"

In [28]:
# hyperparameters
context_window = 4096 #@param
temperature = 0.0 #@param
model_name = 'meta-llama/Llama-2-7b-chat-hf' #@param

In [47]:
from transformers import LlamaTokenizer, LlamaForCausalLM
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model_llm = LlamaForCausalLM.from_pretrained(
        model_name,
        device_map='auto'
    )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [48]:
query_str = documents[3].text
query_str

'http://JUDIS.NIC.IN SUPREME COURT OF INDIA Page 4 of 6 \nyears\' budget and the amount now stands exhausted in \nview of stoppage of the grant by the Central Government.\n        We direct the State of Karnataka as also the State \nof West Bengal to state on oath as to whether they would \nlike to continue with the projects in lieu of the scheme as \nhas been done by the States of Chhattisgarh, Rajasthan \nand Tamil Nadu.  Learned counsel for the Central \nGovernment shall hand over a copy of the project \nadopted by the State of Tamil Nadu, Rajasthan or \nChhattisgarh, as the case may be, to Mr. Tapash Ray, \nlearned senior counsel appearing on behalf of the State of \nWest Bengal and Mr. Sanjay Hegde, learned counsel \nappearing on behalf of the State of Karnataka so as to \nenable them to seek instructions as to whether their States \nare prepared to continue with the said projects on the \nterms adopted by the said States.\n        We would also direct the Government of India to \

In [49]:

def getLLMforPrompts(system_prompt, query_wrapper_prompt) :
    return HuggingFaceLLM(
        context_window=context_window,
        max_new_tokens=256,
        generate_kwargs={"temperature":temperature,"top_p":0.5, "do_sample": False},
        system_prompt= system_prompt,
        query_wrapper_prompt = query_wrapper_prompt,
        tokenizer=tokenizer,
        model=model_llm,
        device_map='auto',
        model_kwargs={"temperature":0.1, "top_p":0.5,"use_auth_token": True}
    )

In [50]:
embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [51]:

def getAllQuestions(response_text):
    j=0
    lines = response_text.split("\n")
    list_questions = []
    for i in range(0, len(lines)):
        if "?" in lines[i] and not lines[i][0].isalpha():
            for j in range(0, len(lines[i])):
                if lines[i][j].isalpha():
                    break
            list_questions.append(lines[i][j:].replace("  "," ").strip())
    if len(list_questions) == 0:
        for i in range(0, len(lines)):
            if "?" in lines[i]:
                list_questions.append(lines[i][j:].replace("  "," ").strip())
    list_questions = list(set(list_questions))
    return list_questions


In [52]:
def getQueryEngine(a_llm, docs):
    service_context = ServiceContext.from_defaults(
        chunk_size=1024,
        llm=a_llm,
        embed_model=embed_model
    )
    index = VectorStoreIndex.from_documents(docs, service_context=service_context)
    return index.as_query_engine()

In [53]:
def create_qa_set(text, docs):
    # print('performing qa for: '+text)
    # print('docs text: '+str(docs))
    qa_pairs = []
    q_llm = getLLMforPrompts(q_system_prompt, query_wrapper_prompt)
    result_llm = q_llm.complete(text).text
    # print(result_llm)
    list_questions = getAllQuestions(result_llm)
    print(" Questions generated for this part - "+str(len(list_questions)))
    # print("questions: "+str(list_questions))
    q_engine = getQueryEngine(getLLMforPrompts(a_system_prompt, query_wrapper_prompt), docs)
    j=1
    for ques in list_questions :
        ans = q_engine.query(ques)
        qa_pairs.append([ques, ans.response.strip()])
        print("  Done with question no. - "+str(j))
        j=j+1
    print(" Q and A pairs for this part - "+str(len(qa_pairs)))
    return qa_pairs

In [54]:
import pandas as pd
#this method takes two pages at a time in a sliding window fashion and generates questions and answers for them
def create_qa_set_for_documents(documents_list):
    all_pairs = []
    for i in range(0, len(documents_list)-1):
        print("Document pair no. - "+str(i))
        text = documents_list[i].text
        if i < len(documents_list)-1 :
            text = text + documents_list[i+1].text
        documents_pair = [documents_list[i].copy()]
        documents_pair[0].text = text
        all_pairs.extend(create_qa_set(text, documents_pair))
        print("Total Pairs so far: "+str(len(all_pairs)))
    return all_pairs

In [87]:
import os
from PyPDF2 import PdfMerger
x = [a for a in os.listdir('../supreme-court-data/data/data/') if a.endswith(".pdf")]
x

['101-1955___jonew__judis__666.pdf',
 '40302-2014___jonew__judis__44742.pdf',
 '77600-1994___jonew__judis__13795.pdf',
 '74656-1991___jonew__judis__17726.pdf',
 '69933-1986___jonew__judis__30235.pdf',
 '31108-2007___jonew__judis__40593.pdf',
 '32467-2011___jonew__judis__39250.pdf',
 '39313-2017___supremecourt_vernacular__2017__39313__39313_2017_6_1501_19107_Judgement_10-Dec-2019_TAM.pdf',
 '-0___jonew__judis__6454.pdf',
 '4125-2004___jonew__judis__26847.pdf',
 '60563-1975___jonew__judis__9537.pdf',
 '8799-2005___jonew__judis__29188.pdf',
 '6191-2019___supremecourt__2019__6191__6191_2019_Judgement_26-Apr-2019.pdf',
 '60549-1999___jonew__judis__16302.pdf',
 '29870-2007___jonew__judis__34902.pdf',
 '31049-2018___supremecourt__2018__31049__31049_2018_Judgement_09-Oct-2018.pdf',
 '60225-1972___jonew__judis__4462.pdf',
 '21743-2011___jonew__judis__38393.pdf',
 '15881-2002___jonew__judis__32587.pdf',
 '75472-1990___jonew__judis__21140.pdf',
 '15154-2002___jonew__judis__19002.pdf',
 '6821-1995

In [89]:
import numpy as np
x = np.random.choice(x, size=50, replace=False)

In [90]:
loader = PDFReader()
data = []
for pdf in x:
    documents = loader.load_data(file=Path('../supreme-court-data/data/data/'+pdf))
    data.append(create_qa_set_for_documents(documents))

invalid pdf header: b'}\xc0\x00\x00%'
incorrect startxref pointer(1)


Document pair no. - 0
 Questions generated for this part - 0
 Q and A pairs for this part - 0
Total Pairs so far: 0
Document pair no. - 1
 Questions generated for this part - 0
 Q and A pairs for this part - 0
Total Pairs so far: 0
Document pair no. - 2
 Questions generated for this part - 0
 Q and A pairs for this part - 0
Total Pairs so far: 0
Document pair no. - 3
 Questions generated for this part - 0
 Q and A pairs for this part - 0
Total Pairs so far: 0
Document pair no. - 4
 Questions generated for this part - 2
  Done with question no. - 1
  Done with question no. - 2
 Q and A pairs for this part - 2
Total Pairs so far: 2
Document pair no. - 5
 Questions generated for this part - 0
 Q and A pairs for this part - 0
Total Pairs so far: 2
Document pair no. - 6
 Questions generated for this part - 0
 Q and A pairs for this part - 0
Total Pairs so far: 2
Document pair no. - 7
 Questions generated for this part - 0
 Q and A pairs for this part - 0
Total Pairs so far: 2
Document pair 

invalid pdf header: b'\x1ca\x00\x00%'
incorrect startxref pointer(1)


  Done with question no. - 10
 Q and A pairs for this part - 10
Total Pairs so far: 39
Document pair no. - 0
 Questions generated for this part - 0
 Q and A pairs for this part - 0
Total Pairs so far: 0
Document pair no. - 1
 Questions generated for this part - 0
 Q and A pairs for this part - 0
Total Pairs so far: 0
Document pair no. - 2
 Questions generated for this part - 0
 Q and A pairs for this part - 0
Total Pairs so far: 0
Document pair no. - 3
 Questions generated for this part - 5
  Done with question no. - 1
  Done with question no. - 2
  Done with question no. - 3
  Done with question no. - 4
  Done with question no. - 5
 Q and A pairs for this part - 5
Total Pairs so far: 5
Document pair no. - 4
 Questions generated for this part - 10
  Done with question no. - 1


KeyboardInterrupt: 

In [99]:
print(data)
len(data)

[                                            Question  \
0  What is the definition of "financial instituti...   
1         What is the Unit Trust of India Act, 1963?   
2  What is the relationship between the Administr...   
3  What is the nature of the power vested in the ...   
4  Is the Central Government responsible for the ...   
5  Can the Debts Recovery Tribunal entertain clai...   
6  Can the Administrator of a specified undertaki...   
7  Is the Specified Company a trustee of the spec...   
8  Can the Administrator of a specified undertaki...   
9  What is the nature of the vesting of the under...   

                                              Answer  
0  The definition of "financial institution" unde...  
1  The Unit Trust of India Act, 1963 is a legisla...  
2  Based on the provided document, the Administra...  
3  The nature of the power vested in the Administ...  
4  No, the Central Government is not responsible ...  
5  Yes, the Debts Recovery Tribunal can entertain...

2

In [101]:
df = pd.DataFrame(data[0], columns = ['Question', 'Answer'])
df2 = pd.DataFrame(data[1], columns = ['Question', 'Answer'])

In [103]:
df = pd.concat([df, df2], ignore_index=True)
df

,Question,Answer
0,"What is the definition of ""financial instituti...","The definition of ""financial institution"" unde..."
1,"What is the Unit Trust of India Act, 1963?","The Unit Trust of India Act, 1963 is a legisla..."
2,What is the relationship between the Administr...,"Based on the provided document, the Administra..."
3,What is the nature of the power vested in the ...,The nature of the power vested in the Administ...
4,Is the Central Government responsible for the ...,"No, the Central Government is not responsible ..."
5,Can the Debts Recovery Tribunal entertain clai...,"Yes, the Debts Recovery Tribunal can entertain..."
6,Can the Administrator of a specified undertaki...,No information available in the provided docum...
7,Is the Specified Company a trustee of the spec...,"Based on the provided document, there is no in..."
8,Can the Administrator of a specified undertaki...,"Yes, the Administrator of a specified undertak..."
9,What is the nature of the vesting of the under...,The nature of the vesting of the undertaking i...


In [104]:
df.to_csv('final_data.csv', index=False, encoding='utf-8')

In [ ]:
# user_queries = ['Seller address in the document?' , 'Client address in the document?', 'seller Tax Id in the document?' ] #@param

# answer = dict()
# for i, user_query in enumerate(user_queries):
#     query_engine = index.as_query_engine()
#     response = query_engine.query(user_query)
#     print(response.response)